In [ ]:
import cv2
import requests
import io
import json
import os
import random
from tqdm import tqdm
import shutil


# ocr 이전 파일 위치
src = '/content/drive/MyDrive/Colab Notebooks/data/telemoji/train/surprise/'    # *수정 필요*   
# ocr 이후 파일 위치
to_dir = '/content/drive/MyDrive/Colab Notebooks/data/fear_filter/'   # 이미지 # *수정 필요*
char_dir = '/content/drive/MyDrive/Colab Notebooks/data/character/'   # 글자

url_api = "https://api.ocr.space/parse/image"
files_ls = os.listdir(src)

# Ocr
for file in tqdm(files_ls): 
    try:
        file_path = os.path.join(src, file)
        img = cv2.imread(file_path) 
        _, compressedimage = cv2.imencode(".jpg", img)  # (Boolean, ndarray)
                                                        # memory buffer에 이미지 넣기
        file_bytes = io.BytesIO(compressedimage)        # memory buffer에 있는 byte 형태 데이터 

        result = requests.post(url_api,
                               files={file_path: file_bytes},
                               data={"apikey": "9f5b13aed588957", # *수정 필요*
                                     "language": "eng"})
        result = result.content.decode() 
        result = json.loads(result)
        parsed_results = result.get("ParsedResults")[0]
        text_detected = parsed_results.get("ParsedText")
        if bool(text_detected) == True:
            print(text_detected)
            shutil.move(file_path, char_dir + file)        # char_dir로 파일 이동
        else:
            shutil.move(file_path, to_dir + file)          # to_dir로 파일 이동
    except (AttributeError, IndexError) as e:
        print(e)
        print(file)

    except TypeError:
        break